# Automatic Spelling Corrector
We will be building an automated spelling corrector using **Hidden Markov Model** of first-order, ie _**bigram**_ dependency assumption.

### Required Imports

In [1]:
from nltk.util import ngrams
from tabulate import tabulate
from collections import defaultdict, Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Preparing the dataset

In [2]:
def read_data(file_name):
    file = open(file_name, 'r')
    data = []
    word = []
    while True:
        line = file.readline()
        if not line:
            break
        line = line.strip()
        line = line.split(',')
        if line[0] == '_':
            data.append(word)
            word = []
            continue
        word.append(line)
    
    return data

train_data = read_data('./train_data.txt')
test_data = read_data('./test_data.txt')        

In [3]:
print(f'Number of train samples : {len(train_data)}')
print(f'Number of test samples : {len(test_data)}')

Number of train samples : 9279
Number of test samples : 1879


### Hidden Markov Model
The model is bigram (first order) without any emission context.

In [4]:
class HMM:
    LAPLACE = 0.0000001
    def __init__(self, train):
        self.state_list = ['*'] + list(set([token[1] for sequence in train for token in sequence])) + ['STOP']
        self.transition = self.generate_transition_matrix(train, ngram=2)
        self.emission = self.generate_emission_matrix(train, with_context=False)
    
    def generate_transition_matrix(self, train, ngram=2, laplace_factor=LAPLACE):
        y = [[token[1] for token in sequence] for sequence in train]
        ngram_tags = []
        for tag_list in y:
            tag_list = ["*"] * (ngram - 1) + tag_list + ["STOP"]
            ngram_tags.extend(ngrams(tag_list, ngram))
        ngram_count = dict(Counter(ngram_tags))

        n_minus_1_gram_tags = []
        for tag_list in y:
            tag_list = ["*"] * (ngram - 1) + tag_list + ["STOP"]
            n_minus_1_gram_tags.extend(ngrams(tag_list, ngram - 1))
        n_minus_1_gram_count = dict(Counter(n_minus_1_gram_tags))

        transition_matrix = defaultdict(lambda: laplace_factor)

        for ngram_tuple in ngram_count:
            n_minus_1_gram_tuple = ngram_tuple[:-1]
            transition_matrix[ngram_tuple] = ngram_count[ngram_tuple] / n_minus_1_gram_count[n_minus_1_gram_tuple]

        return transition_matrix

    def generate_emission_matrix(self, train, with_context=False, laplace_factor=LAPLACE):
        x = [[token[0] for token in sequence] for sequence in train]
        y = [[token[1] for token in sequence] for sequence in train]
        word_tag_count = defaultdict(lambda: 0)
        tag_count = defaultdict(lambda: 0)

        for line, tags in zip(x, y):
            prev_tag = '*'
            for word, tag in zip(line, tags):
                if with_context:
                    tag_count[(tag, prev_tag)] += 1
                    word_tag_count[(word, tag, prev_tag)] += 1
                else:
                    tag_count[(tag,)] += 1
                    word_tag_count[(word, tag)] += 1
                prev_tag = tag
                
        
        emission_matrix = defaultdict(lambda: laplace_factor)
        
        for word_tags in word_tag_count.keys():
            tags = word_tags[1:]
            emission_matrix[word_tags] = word_tag_count[word_tags] / tag_count[tags]

        return emission_matrix

### The Viterbi Algorithm

In [5]:
def kappa(position, state_list):
    return state_list if position not in [0, -1] else ['*']

def viterbi(word, hmm):
    pi = defaultdict(lambda: 0)
    bp = defaultdict(lambda: "OTH")
    pi[(0, '*')] = 1.0
    A = hmm.transition
    B = hmm.emission
    state_list = hmm.state_list

    n = len(word)

    for k in range(1, n + 1):
        u_set = kappa(k - 1, state_list)
        v_set = kappa(k, state_list)

        for v in v_set:
            for u in u_set:
                reach_prob = pi[(k - 1, u)] * A[(u, v)] * B[(word[k - 1], v)]
                if reach_prob > pi[(k, v)]:
                    pi[(k, v)] = reach_prob
                    bp[(k, v)] = u
    
    v_set = kappa(n, state_list)
    result_tags = []
    if n > 0:
        for v in v_set:
            if len(result_tags) == 0:
                result_tags = [v]
            if pi[(n, v)] * A[(v, 'STOP')] > \
            pi[(n, result_tags[0])] * A[(result_tags[0], 'STOP')]:
                result_tags = [v]
    
    for k in range(n - 1, 0, -1):
        result_tags.append(bp[(k + 1, result_tags[-1])])
    
    result_tags.reverse()

    return result_tags

### Testing and Evaluation
We report the overall accuracy of the model and the class-wise (alphabet-wise) precision, recall and f1 scores. The results are tabulated accordingly. 

In [6]:
def evaluate_metrics(true, pred):
    true = [ch for word in true for ch in word]
    pred = [ch for word in pred for ch in word]
    classes = list(set(true))
    classes.sort()
    # accuracy: (tp + tn) / (p + n)
    acc = accuracy_score(true, pred)
    # precision tp / (tp + fp)
    precision = precision_score(true, pred, average=None)
    # recall: tp / (tp + fn)
    recall = recall_score(true, pred, average=None)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(true, pred, average=None)

    return acc, precision, recall, f1, classes

def print_metrics(test_acc, precision, recall, f1, classes):
    print(f"Accuracy of the model: {test_acc}")
    print(tabulate(zip(classes, precision, recall, f1),
                   headers=['Class (Alphabet)', 'Precision', 'Recall', 'F1'],
                   tablefmt='orgtbl'))

def test_and_evaluate(hmm, test_data):
    test = [[token[0] for token in sequence] for sequence in test_data]
    true = [[token[1] for token in sequence] for sequence in test_data]
    pred = []

    for word in test:
        pred.append(viterbi(word, hmm))
    
    predictions_file = open("predictions.txt", "w")
    for predicted_spelling, true_spelling in zip(pred, true):
        for pred_ch, true_ch in zip(predicted_spelling, true_spelling):
            predictions_file.write(f'{pred_ch},{true_ch}\n')
        predictions_file.write(f'_,_\n')

    accuracy, precision, recall, f1, classes = evaluate_metrics(true, pred)
    print_metrics(accuracy, precision, recall, f1, classes)          


In [7]:
hmm = HMM(train_data)
test_and_evaluate(hmm, test_data)

Accuracy of the model: 0.9195359281437125
| Class (Alphabet)   |   Precision |   Recall |       F1 |
|--------------------+-------------+----------+----------|
| a                  |    0.931884 | 0.946981 | 0.939372 |
| b                  |    0.775862 | 0.75     | 0.762712 |
| c                  |    0.943231 | 0.927039 | 0.935065 |
| d                  |    0.92233  | 0.904762 | 0.913462 |
| e                  |    0.922249 | 0.932285 | 0.92724  |
| f                  |    0.836478 | 0.869281 | 0.852564 |
| g                  |    0.921569 | 0.859756 | 0.88959  |
| h                  |    0.923304 | 0.928783 | 0.926036 |
| i                  |    0.918728 | 0.943739 | 0.931065 |
| j                  |    0.888889 | 1        | 0.941176 |
| k                  |    0.893258 | 0.913793 | 0.903409 |
| l                  |    0.912903 | 0.918831 | 0.915858 |
| m                  |    0.931174 | 0.888031 | 0.909091 |
| n                  |    0.908705 | 0.928416 | 0.918455 |
| o           

### Working on the given case of spelling errors
The following sentence (P1) was given with the highlighted wrong words. The corrected words are shown in the sentence P2.  
  
_P1:_ star wars is **ploying** at **thi** regal lloyd **center** and imax multnomah st portland **ang** also at **tho** **centupy** eastport plaza **wuuld** any of **thoss** times **wurk** for **yoz**  
  
_P2:_ star wars is **playing** at **the** regal lloyd **centre** and imax multnomah st portland
**and** also at **the** **century** eastport plaza **would** any of **those** times **work** for **you**  
  
We create a string of these wrong words and evaluate these words according to our viterbi algorithm.

In [8]:
sentence = "ploying thi center ang tho centupy wuuld thoss wurk yoz".split()
correct_sentence = "playing the centre and the century would those work you".split()
sentence = [list(word) for word in sentence]

total_errors = len(correct_sentence)
correct_predictions = 0
for i, word in enumerate(sentence):
    pred_spelling = ''.join(viterbi(word, hmm))
    if correct_sentence[i] == pred_spelling:
        print(f'Corrected Spelling: {pred_spelling}')
        correct_predictions += 1

print(f'Percentage corrected spellings: {correct_predictions * 100 / total_errors}%')
    


Corrected Spelling: century
Corrected Spelling: would
Corrected Spelling: you
Percentage corrected spellings: 30.0%
